<a href="https://colab.research.google.com/github/elainedias16/Data-Mining/blob/main/LeanDL_HPC_Challenge_2025_Atividade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Sobre o Workshop LeanDL-HPC 2025

https://sites.labic.icmc.usp.br/leandl2025/

O LeanDL-HPC 20225 se propõe a explorar e difundir técnicas que permitam **tornar o Deep Learning mais eficiente e escalável no contexto de HPC**, especialmente quando os modelos ultrapassam os recursos típicos dessas infraestruturas. Entre os tópicos em foco estão:

* **Distilação e compressão de modelos**.
* **Quantização**, **pruning**, **cálculos esparços** e **afinamento eficiente de parâmetros**.
* **Otimização de pipelines de inferência** e **deploy consciente de memória/computação**.
* Treinamento e ajuste de LLMs (Modelos de Linguagem de Grande Porte) em sistemas HPC.
* Uso de **arquiteturas leves** como GNNs e CNNs para aplicações científicas.
* Estratégias de **Green AI** e eficiência energética em DL para HPC.



## Este Notebook e o Desafio

Este notebook apresenta **detalhes sobre um dataset** preparado especialmente para um **challenge do workshop LeanDL-HPC 2025**.

O uso deste conjunto de dados é **opcional** e voltado a participantes que desejam trabalhar em uma **tarefa específica** proposta pelo evento.

O desafio envolve **mapear uma amostra teses e dissertações de 2023 para temas estratégicos predefinidos de cada UF**, explorando diferentes níveis de aderência (**BAIXA**, **MÉDIA**, **ALTA**) e, quando possível, apresentando justificativas.

Além de servir como recurso de exploração inicial, o notebook pode apoiar na preparação de soluções para submissão no workshop, alinhadas aos objetivos do LeanDL-HPC 2025.


# LEANDL 2025 — Template de Leitura de Dados e Guia do Desafio

Este notebook apresenta:
- A leitura de **dois arquivos Parquet**:
  - `leandl_oesnpg_dicionario.parquet`: **dicionário de dados** contendo a descrição dos campos.
  - `leandl_oesnpg_dados.parquet`: **dados** contendo amostras de **teses e dissertações de 2023**.
- Um **resumo técnico do desafio**.
- Diretrizes e sugestões de **abordagens de baixo custo** (classificação tradicional, BERT, GNNs, e LLMs até 14B).
- Referências para **medir custo computacional** e **pegada de carbono**.
- Observações importantes sobre a **publicidade** das soluções.



## Desafio: Mapeamento de Teses e Dissertações em Temas Estratégicos da UF

O objetivo é **mapear** cada produção acadêmica (tese/dissertação) para **temas estratégicos predefinidos** do seu estado (UF).

**Observações**:
- Uma mesma produção pode estar associada a **mais de um tema** dentro da mesma UF.
- O mapeamento deve considerar **níveis de aderência**: **BAIXA**, **MÉDIA** e **ALTA**.
- A **justificativa do mapeamento** é **relevante** (explica a decisão) e **bem-vinda**, mas **não é obrigatória**.



## Leitura dos Arquivos Parquet

Abaixo, o código para carregar os dois arquivos:
- `leandl_oesnpg_dicionario.parquet` (dicionário de dados);
- `leandl_oesnpg_dados.parquet` (dados principais).


In [ ]:
#baixando os arquivos (instalar o pacote gdown)
# !gdown 12H957uf6mK-1X_ztT9hgFS1slpN2j-Wh
# !gdown 1-QXkqH8HzLcV2JCA4Nm9G5rQhorYKJVe

Downloading...
From: https://drive.google.com/uc?id=12H957uf6mK-1X_ztT9hgFS1slpN2j-Wh
To: /home/elainedias16/GIT/Data-Mining/leandl_oesnpg_dicionario.parquet
100%|██████████████████████████████████████| 4.24k/4.24k [00:00<00:00, 14.1MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1-QXkqH8HzLcV2JCA4Nm9G5rQhorYKJVe
From (redirected): https://drive.google.com/uc?id=1-QXkqH8HzLcV2JCA4Nm9G5rQhorYKJVe&confirm=t&uuid=963b4921-a3b5-4cbf-8073-f481497545d4
To: /home/elainedias16/GIT/Data-Mining/leandl_oesnpg_dados.parquet
100%|████████████████████████████████████████| 123M/123M [00:01<00:00, 71.1MB/s]


In [2]:
import pandas as pd

# Ajuste os caminhos se necessário

# path_dict = "../elaine/leandl_oesnpg_dicionario.parquet"
# path_data = "../elaine/leandl_oesnpg_dados.parquet"

path_dict = "../../../../nfs/home/elaine/leandl_oesnpg_dicionario.parquet"
path_data = "../../../../nfs/home/elaine/leandl_oesnpg_dados.parquet"

# Leitura usando pandas (requer pyarrow ou fastparquet)
dicionario_df = pd.read_parquet(path_dict)
dados_df = pd.read_parquet(path_data)

print("Dimensões do dicionário:", dicionario_df.shape)
print("Dimensões dos dados:", dados_df.shape)




Dimensões do dicionário: (25, 2)
Dimensões dos dados: (42046, 25)


In [3]:
dicionario_df

,campo,descricao
0,hash_id,Identificador único (hash) para a produção aca...
1,tema_id,Identificador numérico único do tema estratégico.
2,tema,Nome do tema estratégico definido por uma Unid...
3,palavras_chave,Lista de palavras-chave associadas ao tema est...
4,uf_tema_info,Unidade da Federação (UF) responsável pela def...
5,uf_pesquisador,Unidade da Federação (UF) da instituição de ví...
6,nome_programa,Nome do programa de pós-graduação ao qual a pr...
7,sigla_entidade_ensino,Sigla oficial da instituição de ensino respons...
8,nome_producao,Título completo da tese ou dissertação.
9,nome_subtipo_producao,"Tipo de produção acadêmica, como tese (doutora..."



## Níveis de Aderência (BAIXA, MÉDIA, ALTA)

Atribua **níveis de aderência** que reflitam o grau de correspondência entre o **tema estratégico**
e o **conteúdo** da produção (título, resumo, palavras-chave, etc.).

Sugestão de interpretação:
- **ALTA**: forte coerência semântica e conceitual; termos-chave do tema presentes de forma central no trabalho.
- **MÉDIA**: relação indireta ou parcial; termos aparecem com relevância moderada ou secundária.
- **BAIXA**: relação fraca; termos aparecem superficialmente ou em contextos distintos do tema.

Caso produza **explicações** (ex.: `modelo_explicacao`), procure referenciar **trechos** e **termos** que sustentem a decisão.



## Abordagens de Baixo Custo Recomendadas

### 1) Classificadores tradicionais / BERT (binário: **ALTO** vs **NÃO-ALTO**)
- **Embeddings** de título + resumo (ex.: SBERT, sentence-transformers).
- **Similaridade** com descrições dos temas (cosine similarity).
- **k-NN** ou **regressão logística** (binário) para decidir se é **ALTO** ou **NÃO-ALTO**.
- **Fine-tuning BERT** com amostras rotuladas (quando disponíveis), restringindo o problema a **ALTO** vs **NÃO-ALTO** para reduzir custo.

### 2) LLMs até **14B** de parâmetros (com ou sem fine-tuning)
- **Zero-shot** e **few-shot** com **avaliação sistemática de prompts**.
- **Quantização** (ex.: 4-bit) para reduzir memória/latência.
- **Instruções** claras: definir o papel do modelo, a rubrica de decisão (BAIXA/MÉDIA/ALTA) e pedidos de explicação concisa.
- **Auto-avaliação** de confiança e calibragem (ex.: pedir ao modelo que dê um nível e uma justificativa, depois verificar consistência).

### 3) Estratégias baseadas em grafos
- Modelagem de **grafos de similaridade** entre produções e temas.
- **GNNs** (Graph Neural Networks) para propagar sinais de rótulo e robustecer decisões.
- Uso de **k-NN em embeddings** para construir arestas e explorar **comunidades** temáticas.



## Avaliação

As soluções submetidas devem vir com uma **descrição técnica** (preferencialmente um artigo submtido na main track do workshop LeanDL) cobrindo:
1. **Acurácia do mapeamento** por nível (BAIXA/MÉDIA/ALTA) ou na formulação binária (ALTO vs NÃO-ALTO).
2. **Custo computacional**: **tempo** e **memória** consumidos durante inferência/treinamento.
3. (Opcional) **Matriz de confusão** e análises de erro para interpretar decisões.

OBS: quem preferir apresentar em formato de poster, deve submeter apenas um abstract detalhando sua solução e uma cópia do código.

### Medindo Pegada de Carbono
Sugestão: utilizar a biblioteca **codecarbon**.
- Instalação e instruções: https://mlco2.github.io/codecarbon/installation.html



## Observações Finais

- **Origem dos dados**: todas as informações são públicas, obtidas do Portal de Transparência da CAPES e do Observatório da Pós-Graduação da CAPES, com foco em **amostras de teses e dissertações defendidas em 2023**.
- **Publicidade das soluções**: as soluções propostas devem ser **públicas** para garantir a **avaliação** por membros do comitê científico do LeanDL-HPC 2025.


# Considerações Finais para Participação na Challenge do LeanDL-HPC 2025

* **Como artigo regular:** Se você deseja participar da challenge como **artigo regular** da conferência, submeta seu trabalho na **trilha principal (main track)** da conferência, seguindo as diretrizes do sistema de submissão.

* **Como pôster:** Se você deseja participar da challenge no **formato de pôster**, submeta um **documento de até 2 páginas** descrevendo a solução técnica (objetivo, abordagem, arquitetura/modelos, dados, avaliação e limitações).

> **Observação:** A comissão poderá solicitar, para fins de premiação, que um **novo conjunto de teste** seja executado **antes do evento**, de modo a permitir **comparação justa** entre as soluções.


## Solução Proposta

### Criando DataFrame

In [4]:
import numpy as np
import pandas as pd

def to_fset(x):
    if isinstance(x, (list, tuple, np.ndarray)):
        return frozenset(x)
    if pd.isna(x):
        return frozenset()
    #return frozenset([x])
    return frozenset(x)

df_temas = (
    dados_df.assign(palavras_chave=dados_df["palavras_chave"].apply(to_fset))
            [["tema_id", "tema", "uf_tema_info", "nome_producao", "palavras_chave", "descricao_palavra_chave","descricao_resumo", "modelo_nivel" ]]
            .drop_duplicates()
            .dropna()
)


df_temas

,tema_id,tema,uf_tema_info,nome_producao,palavras_chave,descricao_palavra_chave,descricao_resumo,modelo_nivel
0,1,Agronegócio e Tecnologias de Informação e Comu...,ACRE,AS TECNOLOGIAS DIGITAIS DA INFORMAÇÃO E COMUNI...,(tecnologias da informação e comunicação (TIC)...,TECNOLOGIAS DA INFORMAÇÃO E COMUNICAÇÃO;BASE N...,"A PESQUISA, DESCRITA NO PRESENTE TRABALHO, ENV...",BAIXA
1,1,Agronegócio e Tecnologias de Informação e Comu...,ACRE,O CURRÍCULO INTEGRADO DO INSTITUTO FEDERAL DO ...,(tecnologias da informação e comunicação (TIC)...,ENSINO MÉDIO INTEGRADO;ORGANIZAÇÃO CURRICULAR;...,ESTE ESTUDO INVESTIGOU O CURRÍCULO DO ENSINO M...,MEDIA
2,3,Biodiversidade e Biotecnologia,ACRE,TENDÊNCIA TEMPORAL E DISTRIBUIÇÃO ESPACIAL DAS...,"(biotecnologia, desenvolvimento sustentável, c...",EPIDEMIOLOGIA;LEISHMANIA;PRAIS-WINSTEN,"INTRODUÇÃO. AS ANÁLISES NOS BIOMAS: AMAZÔNIA, ...",BAIXA
3,3,Biodiversidade e Biotecnologia,ACRE,MODELAGEM DE BIOMASSA FLORESTAL E CÁLCULO DE C...,"(biotecnologia, desenvolvimento sustentável, c...",GED;SENSORIAMENTO REMOTO;FLORESTA;MAPEAMENTO;B...,DURANTE AS ÚLTIMAS DÉCADAS AS FLORESTAS TROPIC...,MEDIA
4,3,Biodiversidade e Biotecnologia,ACRE,ANÁLISE SOCIOECONÔMICA E AMBIENTAL DA CADEIA P...,"(biotecnologia, desenvolvimento sustentável, c...",BURITI;PRODUTO FLORESTAL NÃO-MADEIEREIRO;COOPE...,O BIOMA AMAZÔNICO OCUPA POSIÇÃO DE DESTAQUE NO...,ALTA
...,...,...,...,...,...,...,...,...
42039,466,Sistemas Agrícolas Sustentáveis,TOCANTINS,AVALIAÇÃO ECOTOXICOLÓGICA DE EFLUENTE DE ABATE...,"(políticas públicas, agricultura familiar, des...",FERTILIZAÇÃO.;INVERTEBRADO DE ÁGUA DOCE;INVERT...,O DESCARTE DE RESÍDUOS EM MATADOUROS É UM DESA...,MEDIA
42040,466,Sistemas Agrícolas Sustentáveis,TOCANTINS,ANÁLISES MORFO-FISIOLÓGICAS DE PLANTAS DE ACRO...,"(políticas públicas, agricultura familiar, des...",MACAÚBA;TOXIDADE;CHIQUIMATO;HERBICIDA;IRGA,AS CULTURAS OLEAGINOSAS SÃO AS MAIS ESTUDADAS ...,BAIXA
42042,467,Transformação Digital,TOCANTINS,DESENVOLVIMENTO DE JOGO EDUCATIVO SOBRE PROPRI...,"(gestão pública, automação de processos, empre...",JOGO DE TABULEIRO;PROPRIEDADE INTELECTUAL;TRAN...,TRATA-SE DE UM PROTÓTIPO PARA A CONCEPÇÃO DE U...,BAIXA
42043,467,Transformação Digital,TOCANTINS,DESAFIOS E OPORTUNIDADES DA IMPLEMENTAÇÃO DA I...,"(gestão pública, automação de processos, empre...",INOVAÇÃO ABERTA;MARCO LEGAL DE CIÊNCIA;TECNOLO...,A INOVAÇÃO ABERTA (IA) TEM SIDO UTILIZADA COMO...,BAIXA


In [5]:
df_temas['modelo_nivel'].isna().any()
df_temas[df_temas['modelo_nivel'].isna()]


,tema_id,tema,uf_tema_info,nome_producao,palavras_chave,descricao_palavra_chave,descricao_resumo,modelo_nivel


## Pré processamento

In [6]:
# import nltk
# nltk.download('punkt')

# !pip install unidecode
# import unidecode

# nltk.download('stopwords')
# from nltk.corpus import stopwords
# stop_words = set(stopwords.words('portuguese'))

# from nltk.tokenize import word_tokenize

# from nltk.stem import RSLPStemmer
# nltk.download('rslp')
# nltk.download('punkt_tab')

In [7]:
# # stemming
# def stemming(text):
#   stemmer = nltk.stem.RSLPStemmer() # stemming to portuguese
#   tokens = word_tokenize(text) #get tokens

#   doc_text_stems = [stemmer.stem(i) for i in tokens]
#   return doc_text_stems

# def remove_stop_words(tokens):
#   text = [token for token in tokens if token not in stop_words]
#   return text

# def remove_non_alphanumeric(tokens):
#   text = [token for token in tokens if token.isalnum()]
#   return text


In [8]:
# def preprocessing(text):

#     text = text.lower()
#     text = unidecode.unidecode(text) #remove accents
#     text = stemming(text) #stemming returns a vector of tokens
#     text = remove_stop_words(text)
#     text = remove_non_alphanumeric(text)
#     return ' '.join(text)


In [9]:
# def preprocess_dataset(df, columns):
#   df[columns] = df[columns].astype(str).applymap(preprocessing)
#   return df

In [10]:
#columns_to_preprocess = ["tema", "palavras_chave", "nome_producao", "descricao_palavra_chave", "descricao_resumo"]
# columns_to_preprocess = ["tema", "descricao_resumo" ]

# df_temas = preprocess_dataset(df_temas, columns_to_preprocess)
# df_temas.head()

## Criação dos embeddings

In [6]:
from codecarbon import EmissionsTracker #to calculate CO2 emissions


#### Embeddings baseados no modelo GloVe

In [7]:
from sentence_transformers import SentenceTransformer

temas = df_temas['tema'].tolist()
resumos = df_temas['descricao_resumo'].tolist()

In [8]:
model_media_glove = SentenceTransformer('mteb-pt/average_pt_nilc_glove_s1000')

In [9]:
# tracker = EmissionsTracker()
# tracker.start()

emb_temas_media_glove = model_media_glove.encode(temas, show_progress_bar=True)
emb_resumo_media_glove = model_media_glove.encode(resumos, show_progress_bar=True)

Batches: 100%|██████████| 1030/1030 [00:06<00:00, 153.77it/s]


#### Embeddings baseados no modelo Word2Vec

In [10]:
model_media_word2vec = SentenceTransformer('mteb-pt/average_pt_nilc_word2vec_skip_s1000')

In [11]:
emb_temas_media_word2vec = model_media_word2vec.encode(temas , show_progress_bar=True)
emb_resumo_media_word2vec = model_media_word2vec.encode(resumos, show_progress_bar=True)

Batches: 100%|██████████| 1030/1030 [00:06<00:00, 155.82it/s]


#### Embeddings baseados em transformers

In [27]:
# MODEL_NAME = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
# MODEL_NAME = 'unicamp-dl/multilingual-MiniLM-L6-v2-pt-msmarco'
MODEL_NAME = 'all-MiniLM-L6-v2'

model_minilm = SentenceTransformer(MODEL_NAME)

emb_temas_minilm = model_minilm.encode(temas, show_progress_bar=True)
emb_resumo_minilm = model_minilm.encode(resumos, show_progress_bar=True)

Batches: 100%|██████████| 1030/1030 [00:39<00:00, 26.14it/s]


### Cálculo Similaridade

In [18]:
from sklearn.metrics.pairwise import cosine_similarity

def calc_cosine_similarity(emb_temas, emb_resumo):
    similarities = []

    for i in range(len(emb_temas)):
        sim = cosine_similarity(emb_temas[i].reshape(1, -1), emb_resumo[i].reshape(1, -1))
        similarities.append(sim[0][0])
    return similarities

In [28]:
sim_word2vec = calc_cosine_similarity(emb_temas_media_word2vec, emb_resumo_media_word2vec)
sim_glove = calc_cosine_similarity(emb_temas_media_glove, emb_resumo_media_glove)
sim_minilm = calc_cosine_similarity(emb_temas_minilm, emb_resumo_minilm)

sim_word2vec = pd.DataFrame(sim_word2vec, columns=["similarity_word2vec"])
sim_glove = pd.DataFrame(sim_glove, columns=["similarity_glove"])
sim_minilm = pd.DataFrame(sim_minilm, columns=["similarity_minilm"])

In [20]:
sim_minilm.head()

,similarity_minilm
0,0.523375
1,0.488582
2,0.502189
3,0.511560
4,0.601142


## Criando o classificador

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder


le = LabelEncoder()
y = le.fit_transform(df_temas['modelo_nivel'])

X_word2vec = sim_word2vec
X_glove = sim_glove
X_minilm = sim_minilm

knn = KNeighborsClassifier(n_neighbors=3)

In [29]:
X_wv_train, X_wv_test, y_wv_train, y_wv_test = train_test_split(sim_word2vec, y, test_size=0.2, random_state=42)

X_gl_train, X_gl_test, y_gl_train, y_gl_test = train_test_split(sim_glove, y, test_size=0.2, random_state=42)

X_minilm_train, X_minilm_test, y_minilm_train, y_minilm_test = train_test_split(sim_minilm, y, test_size=0.2, random_state=42)

knn.fit(X_wv_train, y_wv_train)
y_wv_pred = knn.predict(X_wv_test)

knn.fit(X_gl_train, y_gl_train)
y_gl_pred = knn.predict(X_gl_test)

knn.fit(X_minilm_train, y_minilm_train)
y_minilm_pred = knn.predict(X_minilm_test)

In [30]:
accuracy_wv = accuracy_score(y_wv_test, y_wv_pred)
accuracy_gl = accuracy_score(y_gl_test, y_gl_pred)
accuracy_minilm = accuracy_score(y_minilm_test, y_minilm_pred)

print("Accuracy model based on Word2Vec:", accuracy_wv)
print("Accuracy model based on GloVe:", accuracy_gl)
print("Accuracy model based on MiniLM:", accuracy_minilm)


# emissions: float = tracker.stop()
# print("\n\nTotal de emissões (detalhes em emissions.csv): ",emissions)

Accuracy model based on Word2Vec: 0.30727189919538483
Accuracy model based on GloVe: 0.2964930924548353
Accuracy model based on MiniLM: 0.29770760589039014


### References

@inproceedings{souza2020bertimbau,
  author    = {F{\'a}bio Souza and
               Rodrigo Nogueira and
               Roberto Lotufo},
  title     = {{BERT}imbau: pretrained {BERT} models for {B}razilian {P}ortuguese},
  booktitle = {9th Brazilian Conference on Intelligent Systems, {BRACIS}, Rio Grande do Sul, Brazil, October 20-23 (to appear)},
  year      = {2020}
}
